## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Gatton,AU,-27.5500,152.2833,56.34,62,100,2.46,overcast clouds
1,1,Springdale,US,36.1867,-94.1288,68.49,66,40,9.22,scattered clouds
2,2,Mogadishu,SO,2.0371,45.3438,78.33,84,86,14.03,overcast clouds
3,3,Vestmannaeyjar,IS,63.4427,-20.2734,45.45,87,100,36.08,moderate rain
4,4,Tukrah,LY,32.5341,20.5791,70.32,74,31,20.54,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
95,95,Knysna,ZA,-34.0363,23.0471,57.58,87,58,9.93,light rain
96,96,Bengkulu,ID,-3.8004,102.2655,75.33,86,96,4.05,overcast clouds
97,97,Castro,BR,-24.7911,-50.0119,67.55,69,57,4.92,broken clouds
98,98,Camacha,PT,33.0833,-16.3333,63.82,72,75,13.80,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   130
Country                130
Max Temp               130
Weather Description    130
Lat                    130
Lng                    130
Hotel Name             130
dtype: int64

In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping")
      

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [64]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [73]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacations.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")


In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))